In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import os
import re
import dependency_parser as dp

In [34]:
with open('../hindi.input.txt') as f:
    source_text = f.readlines()

In [35]:
source_text

['हिन्दी संवैधानिक रूप से भारत की प्रथम राजभाषा और भारत की सबसे अधिक बोली और समझी जाने वाली भाषा है। चीनी के बाद यह विश्व में सबसे अधिक बोली जाने वाली भाषा भी है।\n',
 '\n',
 'हिन्दी और इसकी बोलियाँ उत्तर एवं मध्य भारत के विविध राज्यों में बोली जाती हैं। भारत और अन्य देशों में ६० करोड़ से अधिक लोग हिन्दी बोलते, पढ़ते और लिखते हैं। फ़िजी, मॉरिशस, गयाना, सूरीनाम की अधिकतर और नेपाल की कुछ जनता हिन्दी बोलती है।\n',
 '\n',
 'हिन्दी राष्ट्रभाषा, राजभाषा, सम्पर्क भाषा, जनभाषा के सोपानों को पार कर विश्वभाषा बनने की ओर अग्रसर है। भाषा विकास क्षेत्र से जुड़े वैज्ञानिकों की भविष्यवाणी हिन्दी प्रेमियों के लिए बड़ी सन्तोषजनक है कि आने वाले समय में विश्वस्तर पर अन्तर्राष्ट्रीय महत्त्व की जो चन्द भाषाएँ होंगी उनमें हिन्दी भी प्रमुख होगी।\n',
 '\n',
 "हिन्दी शब्द का सम्बन्ध संस्कृत शब्द सिन्धु से माना जाता है। 'सिन्धु' सिन्ध नदी को कहते थे ओर उसी आधार पर उसके आस-पास की भूमि को सिन्धु कहने लगे। यह सिन्धु शब्द ईरानी में जाकर ‘हिन्दू’, हिन्दी और फिर ‘हिन्द’ हो गया। बाद में ईरानी धीरे-धीरे भारत के अधिक भा

In [68]:
input_file = '../hindi.input.txt'
output_file = '../hindi.output.txt'
Subs = {
    ',':'\n,\n',
    '(': '\n(\n',
    ')': '\n)\n',
    '‘': '\n‘\n',
    '’': '\n’\n',
    '"': '\n"\n',
    "'": "\n'\n",
    "'": "\n'\n",
    '+': '\n+\n',
}
rep = dict((re.escape(k), v) for k, v in Subs.items())
pattern = re.compile("|".join(rep.keys()))
temporary_tokenized_normalized_filepath = 'refactor/temp.txt'
temporary_conll_filepath = 'refactor/temp.conll'
jarpath = 'bin/malt.jar'
multi_tabs = re.compile('\t+')
with open('hindi.lemma.json') as lemmafile:
    loaded_lemma = json.load(lemmafile)

# Start doing this
tokenized = (dp.normalize(dp.tokenise(x, language='Hindi')) for x in source_text) 
flat = (x for y in tokenized for x in y if x!='\n')
new_tokens = (pattern.sub(lambda m: rep[re.escape(m.group(0))], text) for text in flat)
sentence_splits = (tokens.split('\n') for tokens in new_tokens)
flatten_sentence_splits = (x for y in sentence_splits for x in y if x)
if os.path.exists('temp.txt'):
    os.unlink('temp.txt')
with open('temp.txt', 'w')as f:
    f.writelines((f'{x}\n' for x in flatten_sentence_splits))

lemma_input = (multi_tabs.sub('\t', tokens) for tokens in dp.get_pos_tags(temporary_tokenized_normalized_filepath))
lemmatized = dp.lemmatise(lemma_input, loaded_lemma)
vert_tags = dp.tag2vert(lemmatized)
mod_pos = dp.modify_pos_tags(vert_tags)
cut_tokens = (token.split('\t')[:3] for token in mod_pos)
dp.to_conll(cut_tokens, 'temp.conll')
sub = dp.run_malt_parser(temporary_conll_filepath, jarpath)
with open(output_file, 'w') as f:
    f.writelines((f'{x}\n' for x in sub))

In [66]:
sub = dp.run_malt_parser(temporary_conll_filepath, jarpath)


In [67]:
for line in sub:
    print(line)

1	हिन्दी	हिन्दी	NN	0	ROOT
2	संवैधानिक	संवैधानिक	JJ	3	nmod__adj
3	रूप	रूप	NN	0	ROOT
4	से	से	PSP:से	3	lwg__psp
5	भारत	भारत	NNP	8	r6
6	की	का	PSP:का	5	lwg__psp
7	प्रथम	प्रथम	QO	8	nmod__adj
8	राजभाषा	राजभाषा	NN	9	ccof
9	और	और	CC:और	0	ROOT
10	भारत	भारत	NNP	14	r6
11	की	का	PSP:का	10	lwg__psp
12	सबसे	सबसे	INTF	14	jjmod__intf
13	अधिक	अधिक	QF	14	nmod__adj
14	बोली	बोल	VM	9	ccof
15	और	और	CC:और	9	ccof
16	समझी	समझ	VM	19	nmod
17	जाने	जान	VAUX	16	lwg__vaux
18	वाली	वाली	PSP:वाली	16	lwg__psp
19	भाषा	भाषा	NN	20	k1s
20	है	है	VM	15	ccof
21	.	.	.	20	rsym

1	चीनी	चीनी	NN	14	k7t
2	के	का	PSP:का	1	lwg__psp
3	बाद	बाद	NST	1	lwg__psp
4	यह	यह	DEM	5	nmod__adj
5	विश्व	विश्व	NN	9	k7p
6	में	में	PSP:में	5	lwg__psp
7	सबसे	सबसे	INTF	9	vmod
8	अधिक	अधिक	QF	7	nmod__adj
9	बोली	बोल	VM	12	nmod__relc
10	जाने	जान	VAUX	9	lwg__vaux
11	वाली	वाली	PSP:वाली	9	lwg__psp
12	भाषा	भाषा	NN	14	k1
13	भी	भी	RP	12	lwg__rp
14	है	है	VM	0	main
15	.	.	.	14	rsym

1	हिन्दी	हिन्दी	NN	2	ccof
2	और	और	CC:और	13	k7p
3	इसकी	इसकी	PRP	4	r6
4	बोलियां	बोलियां	NN	

In [5]:
tokenized = [dp.tokenise(x, language='Hindi') for x in source_text];
# tokenized will be a list of sentences
normalized = [dp.normalize(x) for x in tokenized]
flat = [x for y in normalized for x in y if x !='\n']
Subs = {
    ',':'\n,\n',
    '(': '\n(\n',
    ')': '\n)\n',
    '‘': '\n‘\n',
    '’': '\n’\n',
    '"': '\n"\n',
    "'": "\n'\n",
    "'": "\n'\n",
    '+': '\n+\n',
}
rep = dict((re.escape(k), v) for k, v in Subs.items())
pattern = re.compile("|".join(rep.keys()))
new_tokens = [pattern.sub(lambda m: rep[re.escape(m.group(0))], text) for text in flat]
new_tokens = [tokens.split('\n') for tokens in new_tokens]
new_tokens = [ x for y in new_tokens for x in y if x]
new_tokens
with open('temp.txt', 'a') as f:
    f.writelines((f'{x}\n' for x in new_tokens))
filepath = 'refactor/temp.txt'

In [7]:
filepath = 'refactor/temp.txt'

In [8]:
multi_tabs = re.compile('\t+')
lemma_input = [multi_tabs.sub('\t', tokens) for tokens in dp.get_pos_tags(filepath)];
lemma_input

['हिन्दी\tNN.n.f.sg.3.d',
 'संवैधानिक\tJJ.adj.any.any..any',
 'रूप\tNN.n.m.sg.3.d',
 'से\tPSP.v.any.any.any.',
 'भारत\tNNP.n.m.sg.3.d',
 'की\tPSP.psp.f.sg..d',
 'प्रथम\tQO.adj.any.any..any',
 'राजभाषा\tNN.n.f.sg.3.d',
 'और\tCC.adv....',
 'भारत\tNNP.n.m.sg.3.d',
 'की\tPSP.psp.f.sg..d',
 'सबसे\tINTF.adj.any.any..any',
 'अधिक\tQF.adv....',
 'बोली\tVM.v.f.sg.any.',
 'और\tCC.adv....',
 'समझी\tVM.v.f.sg.any.',
 'जाने\tVAUX.v.any.sg.2.',
 'वाली\tPSP.n.f.sg.3.d',
 'भाषा\tNN.n.f.sg.3.d',
 'है\tVM.v.any.sg.2.',
 '.\tSYM.punc....',
 '</s>',
 '<s>',
 'चीनी\tNN.adj.f.any..any',
 'के\tPSP.psp.m.sg..o',
 'बाद\tNST.adv....',
 'यह\tDEM.pn.any.sg.3.d',
 'विश्व\tNN.n.m.sg.3.d',
 'में\tPSP.psp....',
 'सबसे\tINTF.adj.any.any..any',
 'अधिक\tQF.adv....',
 'बोली\tVM.v.f.sg.any.',
 'जाने\tVAUX.v.any.sg.2.',
 'वाली\tPSP.n.f.sg.3.d',
 'भाषा\tNN.n.f.sg.3.d',
 'भी\tRP.adv....',
 'है\tVM.v.any.sg.2.',
 '.\tSYM.punc....',
 '</s>',
 '<s>',
 'हिन्दी\tNN.n.f.sg.3.d',
 'और\tCC.adv....',
 'इसकी\tPRP.pn.f.sg.3.d',
 'बोलिय

In [9]:
with open('../hindi.output.tmp.tags.expectedlemma') as f:
    gold2 = f.readlines()
gold2

['हिन्दी\tNN.n.f.sg.3.d\n',
 'संवैधानिक\tJJ.adj.any.any..any\n',
 'रूप\tNN.n.m.sg.3.d\n',
 'से\tPSP.v.any.any.any.\n',
 'भारत\tNNP.n.m.sg.3.d\n',
 'की\tPSP.psp.f.sg..d\n',
 'प्रथम\tQO.adj.any.any..any\n',
 'राजभाषा\tNN.n.f.sg.3.d\n',
 'और\tCC.adv....\n',
 'भारत\tNNP.n.m.sg.3.d\n',
 'की\tPSP.psp.f.sg..d\n',
 'सबसे\tINTF.adj.any.any..any\n',
 'अधिक\tQF.adv....\n',
 'बोली\tVM.v.f.sg.any.\n',
 'और\tCC.adv....\n',
 'समझी\tVM.v.f.sg.any.\n',
 'जाने\tVAUX.v.any.sg.2.\n',
 'वाली\tPSP.n.f.sg.3.d\n',
 'भाषा\tNN.n.f.sg.3.d\n',
 'है\tVM.v.any.sg.2.\n',
 '.\tSYM.punc....\n',
 '</s>\n',
 '<s>\n',
 'चीनी\tNN.adj.f.any..any\n',
 'के\tPSP.psp.m.sg..o\n',
 'बाद\tNST.adv....\n',
 'यह\tDEM.pn.any.sg.3.d\n',
 'विश्व\tNN.n.m.sg.3.d\n',
 'में\tPSP.psp....\n',
 'सबसे\tINTF.adj.any.any..any\n',
 'अधिक\tQF.adv....\n',
 'बोली\tVM.v.f.sg.any.\n',
 'जाने\tVAUX.v.any.sg.2.\n',
 'वाली\tPSP.n.f.sg.3.d\n',
 'भाषा\tNN.n.f.sg.3.d\n',
 'भी\tRP.adv....\n',
 'है\tVM.v.any.sg.2.\n',
 '.\tSYM.punc....\n',
 '</s>\n',
 '<s>\n'

lemmaDict= {}  # word : { pos : lemma}
def loadLemmatiser(file):
    for line in open(file):
        line=line.strip()
        word= line.split('\t')[0]
        tags= line.split('\t')[1:]
        if word not in lemmaDict:
            lemmaDict[word]= {}
        for t in tags:
            (tag, lemma)= t.split(' ')
            if tag in lemmaDict[word]:
                continue
            if lemma.strip()!="":
                lemmaDict[word][tag]= lemma

In [10]:
import json
# import gramex.cache as gc

In [11]:
# import yaml
# with open('hindi.lemma.yaml') as yf:
#     loaded_lemma = yaml.load(yf)

In [12]:
# with open('hindi.lemma.json', 'w') as yf:
#     json.dump(loaded_lemma, yf)

In [13]:
with open('hindi.lemma.json') as lemmafile:
    loaded_lemma = json.load(lemmafile)
# loaded_lemma = gc.open('hindi.lemma.json')

In [14]:
lemmatized = dp.lemmatise(lemma_input, loaded_lemma)

In [15]:
vert_tags = dp.tag2vert(lemmatized)

In [16]:
mod_pos = dp.modify_pos_tags(vert_tags)

In [17]:
with open('../@.tmp.pos') as f:
    gold_pos = f.readlines()
gold_pos = [line.strip() for line in gold_pos]
for actual, expected in zip(mod_pos, gold_pos):
    if actual != expected:
        print(f'actual {actual} and expected {expected}')

actual से	PSP:से	से-p	0	v	any	any	any	 and expected से	PSP:से	से-p	0	v	any	any	any
actual और	CC:और	और-c		adv				 and expected और	CC:और	और-c		adv
actual अधिक	QF	अधिक-x		adv				 and expected अधिक	QF	अधिक-x		adv
actual बोली	VM	बोल-v	या	v	f	sg	any	 and expected बोली	VM	बोल-v	या	v	f	sg	any
actual और	CC:और	और-c		adv				 and expected और	CC:और	और-c		adv
actual समझी	VM	समझ-v	या	v	f	sg	any	 and expected समझी	VM	समझ-v	या	v	f	sg	any
actual जाने	VAUX	जान-v	एं	v	any	sg	2	 and expected जाने	VAUX	जान-v	एं	v	any	sg	2
actual है	VM	है-v	है	v	any	sg	2	 and expected है	VM	है-v	है	v	any	sg	2
actual .	.	.-x		punc				 and expected .	.	.-x		punc
actual बाद	NST	बाद-n		adv				 and expected बाद	NST	बाद-n		adv
actual में	PSP:में	में-p		psp				 and expected में	PSP:में	में-p		psp
actual अधिक	QF	अधिक-x		adv				 and expected अधिक	QF	अधिक-x		adv
actual बोली	VM	बोल-v	या	v	f	sg	any	 and expected बोली	VM	बोल-v	या	v	f	sg	any
actual जाने	VAUX	जान-v	एं	v	any	sg	2	 and expected जाने	VAUX	जान-v	एं	v	any	sg	2
actual भी	RP	भी-x

In [18]:
cut_tokens = [token.split('\t')[:3] for token in mod_pos]
cut_tokens

[['हिन्दी', 'NN', 'हिन्दी-n'],
 ['संवैधानिक', 'JJ', 'संवैधानिक-j'],
 ['रूप', 'NN', 'रूप-n'],
 ['से', 'PSP:से', 'से-p'],
 ['भारत', 'NNP', 'भारत-n'],
 ['की', 'PSP:का', 'का-p'],
 ['प्रथम', 'QO', 'प्रथम-x'],
 ['राजभाषा', 'NN', 'राजभाषा-n'],
 ['और', 'CC:और', 'और-c'],
 ['भारत', 'NNP', 'भारत-n'],
 ['की', 'PSP:का', 'का-p'],
 ['सबसे', 'INTF', 'सबसे-x'],
 ['अधिक', 'QF', 'अधिक-x'],
 ['बोली', 'VM', 'बोल-v'],
 ['और', 'CC:और', 'और-c'],
 ['समझी', 'VM', 'समझ-v'],
 ['जाने', 'VAUX', 'जान-v'],
 ['वाली', 'PSP:वाली', 'वाली-p'],
 ['भाषा', 'NN', 'भाषा-n'],
 ['है', 'VM', 'है-v'],
 ['.', '.', '.-x'],
 ['</s>'],
 ['<s>'],
 ['चीनी', 'NN', 'चीनी-n'],
 ['के', 'PSP:का', 'का-p'],
 ['बाद', 'NST', 'बाद-n'],
 ['यह', 'DEM', 'यह-x'],
 ['विश्व', 'NN', 'विश्व-n'],
 ['में', 'PSP:में', 'में-p'],
 ['सबसे', 'INTF', 'सबसे-x'],
 ['अधिक', 'QF', 'अधिक-x'],
 ['बोली', 'VM', 'बोल-v'],
 ['जाने', 'VAUX', 'जान-v'],
 ['वाली', 'PSP:वाली', 'वाली-p'],
 ['भाषा', 'NN', 'भाषा-n'],
 ['भी', 'RP', 'भी-x'],
 ['है', 'VM', 'है-v'],
 ['.', '.', '.-x'

In [19]:
index = 1
with open('temp.conll', 'w+') as f:
    for tokens in cut_tokens:
        if len(tokens) > 2:
            j=[ str(index), tokens[0], tokens[2][:-2], '_', tokens[1], '_|_|_|_|_', '_', '_', '_', '_' ];
            f.write('\t'.join(j)+"\n");
            index = index + 1;
        else:
            index = 1;
            f.write("\n");

In [20]:
with open('temp.conll') as f:
    connll = f.readlines()

In [21]:
connll

['1\tहिन्दी\tहिन्दी\t_\tNN\t_|_|_|_|_\t_\t_\t_\t_\n',
 '2\tसंवैधानिक\tसंवैधानिक\t_\tJJ\t_|_|_|_|_\t_\t_\t_\t_\n',
 '3\tरूप\tरूप\t_\tNN\t_|_|_|_|_\t_\t_\t_\t_\n',
 '4\tसे\tसे\t_\tPSP:से\t_|_|_|_|_\t_\t_\t_\t_\n',
 '5\tभारत\tभारत\t_\tNNP\t_|_|_|_|_\t_\t_\t_\t_\n',
 '6\tकी\tका\t_\tPSP:का\t_|_|_|_|_\t_\t_\t_\t_\n',
 '7\tप्रथम\tप्रथम\t_\tQO\t_|_|_|_|_\t_\t_\t_\t_\n',
 '8\tराजभाषा\tराजभाषा\t_\tNN\t_|_|_|_|_\t_\t_\t_\t_\n',
 '9\tऔर\tऔर\t_\tCC:और\t_|_|_|_|_\t_\t_\t_\t_\n',
 '10\tभारत\tभारत\t_\tNNP\t_|_|_|_|_\t_\t_\t_\t_\n',
 '11\tकी\tका\t_\tPSP:का\t_|_|_|_|_\t_\t_\t_\t_\n',
 '12\tसबसे\tसबसे\t_\tINTF\t_|_|_|_|_\t_\t_\t_\t_\n',
 '13\tअधिक\tअधिक\t_\tQF\t_|_|_|_|_\t_\t_\t_\t_\n',
 '14\tबोली\tबोल\t_\tVM\t_|_|_|_|_\t_\t_\t_\t_\n',
 '15\tऔर\tऔर\t_\tCC:और\t_|_|_|_|_\t_\t_\t_\t_\n',
 '16\tसमझी\tसमझ\t_\tVM\t_|_|_|_|_\t_\t_\t_\t_\n',
 '17\tजाने\tजान\t_\tVAUX\t_|_|_|_|_\t_\t_\t_\t_\n',
 '18\tवाली\tवाली\t_\tPSP:वाली\t_|_|_|_|_\t_\t_\t_\t_\n',
 '19\tभाषा\tभाषा\t_\tNN\t_|_|_|_|_\t_\t_\t_\t_\n',
 '20\tहै\tहै\

In [22]:
with open('../hindi.output.tmp.tag.conll') as f:
    expected_connl = f.readlines()

In [23]:
for ind, (actual,expected) in enumerate(zip(connll, expected_connl)):
    if actual != expected:
        print(f'{actual}***************{expected}')

In [24]:
input_filepath = 'refactor/temp.conll'
jarpath = 'bin/malt.jar'

In [43]:
sub = dp.run_malt_parser(input_filepath, jarpath)

In [44]:
sub

['1\tहिन्दी\tहिन्दी\t_\tNN\t_|_|_|_|_\t0\tROOT\t_\t_',
 '2\tसंवैधानिक\tसंवैधानिक\t_\tJJ\t_|_|_|_|_\t3\tnmod__adj\t_\t_',
 '3\tरूप\tरूप\t_\tNN\t_|_|_|_|_\t0\tROOT\t_\t_',
 '4\tसे\tसे\t_\tPSP:से\t_|_|_|_|_\t3\tlwg__psp\t_\t_',
 '5\tभारत\tभारत\t_\tNNP\t_|_|_|_|_\t8\tr6\t_\t_',
 '6\tकी\tका\t_\tPSP:का\t_|_|_|_|_\t5\tlwg__psp\t_\t_',
 '7\tप्रथम\tप्रथम\t_\tQO\t_|_|_|_|_\t8\tnmod__adj\t_\t_',
 '8\tराजभाषा\tराजभाषा\t_\tNN\t_|_|_|_|_\t9\tccof\t_\t_',
 '9\tऔर\tऔर\t_\tCC:और\t_|_|_|_|_\t0\tROOT\t_\t_',
 '10\tभारत\tभारत\t_\tNNP\t_|_|_|_|_\t14\tr6\t_\t_',
 '11\tकी\tका\t_\tPSP:का\t_|_|_|_|_\t10\tlwg__psp\t_\t_',
 '12\tसबसे\tसबसे\t_\tINTF\t_|_|_|_|_\t14\tjjmod__intf\t_\t_',
 '13\tअधिक\tअधिक\t_\tQF\t_|_|_|_|_\t14\tnmod__adj\t_\t_',
 '14\tबोली\tबोल\t_\tVM\t_|_|_|_|_\t9\tccof\t_\t_',
 '15\tऔर\tऔर\t_\tCC:और\t_|_|_|_|_\t9\tccof\t_\t_',
 '16\tसमझी\tसमझ\t_\tVM\t_|_|_|_|_\t19\tnmod\t_\t_',
 '17\tजाने\tजान\t_\tVAUX\t_|_|_|_|_\t16\tlwg__vaux\t_\t_',
 '18\tवाली\tवाली\t_\tPSP:वाली\t_|_|_|_|_\t16\tlwg__psp\t_\t_',


In [26]:
out = sub.stdout.decode('utf-8').split('\n')

In [27]:
def parse_malt_output_helper(token):
    list_ = token.split('\t')
    if len(list_) > 2:
        return (list_[0], list_[1], list_[2], list_[4], list_[6], list_[7])

In [28]:
output = [parse_malt_output_helper(token) for token in out]

In [29]:
output

[('1', 'हिन्दी', 'हिन्दी', 'NN', '0', 'ROOT'),
 ('2', 'संवैधानिक', 'संवैधानिक', 'JJ', '3', 'nmod__adj'),
 ('3', 'रूप', 'रूप', 'NN', '0', 'ROOT'),
 ('4', 'से', 'से', 'PSP:से', '3', 'lwg__psp'),
 ('5', 'भारत', 'भारत', 'NNP', '8', 'r6'),
 ('6', 'की', 'का', 'PSP:का', '5', 'lwg__psp'),
 ('7', 'प्रथम', 'प्रथम', 'QO', '8', 'nmod__adj'),
 ('8', 'राजभाषा', 'राजभाषा', 'NN', '9', 'ccof'),
 ('9', 'और', 'और', 'CC:और', '0', 'ROOT'),
 ('10', 'भारत', 'भारत', 'NNP', '14', 'r6'),
 ('11', 'की', 'का', 'PSP:का', '10', 'lwg__psp'),
 ('12', 'सबसे', 'सबसे', 'INTF', '14', 'jjmod__intf'),
 ('13', 'अधिक', 'अधिक', 'QF', '14', 'nmod__adj'),
 ('14', 'बोली', 'बोल', 'VM', '9', 'ccof'),
 ('15', 'और', 'और', 'CC:और', '9', 'ccof'),
 ('16', 'समझी', 'समझ', 'VM', '19', 'nmod'),
 ('17', 'जाने', 'जान', 'VAUX', '16', 'lwg__vaux'),
 ('18', 'वाली', 'वाली', 'PSP:वाली', '16', 'lwg__psp'),
 ('19', 'भाषा', 'भाषा', 'NN', '20', 'k1s'),
 ('20', 'है', 'है', 'VM', '15', 'ccof'),
 ('21', '.', '.', '.', '20', 'rsym'),
 None,
 ('1', 'चीनी',